In [2]:
import tensorflow as tf
from DoVaeSupplementary import *
from DoSplitData import load_split_data
from DoSupplementaryTrain import *
from tensorflow.keras.callbacks import LearningRateScheduler
from CBAM_attention3D import *

In [4]:
data_pathX = 'FileMatrices_XNormalWhtSDummies180.csv'
data_pathY = 'FileMatrices_YNormalWhtSDummies180.csv'
input_shape = (40, 40, 1)
batch_size = 64
kernel_size = 3
filters = 16
latent_dim = 100
epochs = 1000
use_mse = True
load_weights = False
h5model = ['attmodel'+'.h5']
name_model = '3DNetv1AttentionDrop'
time_callback = TimeHistory()
for data_train_models in load_split_data (data_pathX,data_pathY,batch_size,0.10,4,'rnn'):
    datamd = data_train_models

Laoding the sensor data
2432


In [6]:
train_data_modelrnn = datamd.model_train_data

In [7]:
x_train_normal_ns = train_data_modelrnn.Xtran
x_train_normal_n = train_data_modelrnn.Xtran
x_val_nomal_ns = train_data_modelrnn.Xval
x_val_nomal_n = train_data_modelrnn.Xval
yVal = train_data_modelrnn.yVal

In [8]:
encoder_inputs = tf.keras.Input(shape=(4,40, 40, 1))
#x = tf.keras.layers.Dropout(0.25)(encoder_inputs)
x = tf.keras.layers.Conv3D(32, kernel_size=(3, 3, 3), strides=1, activation='relu',padding="same")(encoder_inputs)
x = tf.keras.layers.Dropout(0.25)(x)
x = cbam_block(x)
x = tf.keras.layers.Conv3D(64, kernel_size=(3, 3, 3), strides=2,  activation='relu', padding="same")(x)
x = tf.keras.layers.Dropout(0.25)(x)
x = cbam_block(x)
x = tf.keras.layers.Conv3D(128, kernel_size=(3, 3, 3), strides=2,  activation='relu', padding="same")(x)
x = tf.keras.layers.Dropout(0.25)(x)
x = cbam_block(x)
x = tf.keras.layers.Conv3D(256, kernel_size=(3, 3, 3), strides=2,  activation='relu', padding="same")(x)
x = tf.keras.layers.Dropout(0.25)(x)
x = cbam_block(x)
x = tf.keras.layers.Flatten()(x)
z_mean = tf.keras.layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = tf.keras.layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = tf.keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")

In [9]:
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 4, 40, 40,   0           []                               
                                1)]                                                               
                                                                                                  
 conv3d (Conv3D)                (None, 4, 40, 40, 3  896         ['input_1[0][0]']                
                                2)                                                                
                                                                                                  
 dropout (Dropout)              (None, 4, 40, 40, 3  0           ['conv3d[0][0]']                 
                                2)                                                          

In [10]:
latent_inputs = tf.keras.Input(shape=(latent_dim,))
x = tf.keras.layers.Dense(128,activation=tf.nn.relu)(latent_inputs)
x = tf.keras.layers.Dense(1*10* 10 *128,activation=tf.nn.relu)(x)
x = tf.keras.layers.Reshape((1,10,10, 128))(x)
x = tf.keras.layers.Conv3DTranspose(256, kernel_size=(3, 3, 3), strides=1, activation='relu', padding="same")(x)
x = tf.keras.layers.Dropout(0.25)(x)
x = tf.keras.layers.Conv3DTranspose(128,  kernel_size=(3, 3, 3), strides=2, activation='relu', padding="same")(x)
x = tf.keras.layers.Dropout(0.25)(x)
x = tf.keras.layers.Conv3DTranspose(64,  kernel_size=(3, 3, 3), strides=2, activation='relu', padding="same")(x)
x = tf.keras.layers.Dropout(0.25)(x)
x = tf.keras.layers.Conv3DTranspose(32,  kernel_size=(3, 3, 3), strides=1, activation='relu', padding="same")(x) 
x = tf.keras.layers.Dropout(0.25)(x)
x = tf.keras.layers.Conv3DTranspose(1,  kernel_size=(3, 3, 3), strides=1, activation='relu', padding="same")(x)
x = tf.keras.layers.Dropout(0.25)(x)
decoder = tf.keras.Model(latent_inputs, x, name="decoder")
mvae = VAE_TD(encoder, decoder) 

In [11]:
decoder.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 100)]             0         
                                                                 
 dense (Dense)               (None, 128)               12928     
                                                                 
 dense_1 (Dense)             (None, 12800)             1651200   
                                                                 
 reshape (Reshape)           (None, 1, 10, 10, 128)    0         
                                                                 
 conv3d_transpose (Conv3DTra  (None, 1, 10, 10, 256)   884992    
 nspose)                                                         
                                                                 
 dropout_4 (Dropout)         (None, 1, 10, 10, 256)    0         
                                                           

In [12]:
mvae.compile(optimizer=tf.keras.optimizers.Adam())

In [13]:
history = mvae.fit(x_train_normal_ns,
                   epochs=epochs, 
                   batch_size=batch_size,
                   callbacks=[EarlyStoppingAtMinLoss(),LossHistory(),time_callback]).history

Epoch 1/1000
10/10 [==============================] - 88s 7s/step - loss: 180.8981
Epoch 2/1000
10/10 [==============================] - 89s 9s/step - loss: 188.6523
Epoch 3/1000
10/10 [==============================] - 71s 7s/step - loss: 171.5430
Epoch 4/1000
10/10 [==============================] - 66s 7s/step - loss: 119.0523
Epoch 5/1000
10/10 [==============================] - 64s 6s/step - loss: 103.6202
Epoch 6/1000
10/10 [==============================] - 85s 9s/step - loss: 99.8200
Epoch 7/1000
10/10 [==============================] - 68s 6s/step - loss: 86.1277
Epoch 8/1000
10/10 [==============================] - 63s 6s/step - loss: 68.0488
Epoch 9/1000
10/10 [==============================] - 62s 6s/step - loss: 55.8853
Epoch 10/1000
10/10 [==============================] - 66s 6s/step - loss: 44.3493
Epoch 11/1000
10/10 [==============================] - 71s 7s/step - loss: 33.4958
Epoch 12/1000
10/10 [==============================] - 86s 8s/step - loss: 26.5709
Epoch 13